In [1]:
import mitsuba
import numpy as np
import os


import multiprocessing
#import igl
import numpy as np
import matplotlib.pyplot as plt
import patterns as pa
import matplotlib


# TODOs
# 1. Batchrender folder: one job per object
# 1.1 Render object, multiple rotations per object, one pattern
# 2. Cleanup scenes <include filename="nested-scene.xml"/>
# 3. Cleanup config

In [117]:
import json
import yaml
import os
import glob
from PIL import Image
import numpy as np
import mitsuba
import multiprocessing
from mitsuba.core import * 
from mitsuba.render import Scene, Texture
from mitsuba.render import RenderQueue,RenderJob
from mitsuba.render import SceneHandler



def render_object(config_path):
    assert os.path.isfile(config_path)
    
    with open(config_path, "r") as fi:
        parameters = json.load(fi)
    
    patterns = prepare_patterns(parameters)
    rotations = prepare_rotations(parameters)
    print(len(patterns), len(rotations), rotations)
    
    render_all_patterns_and_rotations(parameters, patterns, rotations)

    
def prepare_rotations(parameters):
    if parameters["rot_type"] == "Turntable":
        rotations = [] 
        rots = np.linspace(parameters["rot_range"][0], parameters["rot_range"][1], parameters["rot_range"][2], endpoint=False)
        for i in range(len(rots)):
            rotations.append({"name": "rot_%03i"%i, "rot_x": parameters["rot_x"], "rot_y": rots[i], "rot_z": parameters["rot_z"]})
    elif parameters["rot_type"] == "Random":
        rotations = []
        for i in range(parameters["rot_range"][3]):
            rot_x = np.random.randint(parameters["rot_range"][0][0], parameters["rot_range"][0][1])
            rot_y = np.random.randint(parameters["rot_range"][1][0], parameters["rot_range"][1][1])
            rot_z = np.random.randint(parameters["rot_range"][2][0], parameters["rot_range"][2][1])
            rotations.append({"name": "rot_%03i"%i, "rot_x": rot_x, "rot_y": rot_y, "rot_z": rot_z})
    elif parameters["rot_type"] == "Fixed":
        rotations = [{"name": "rot_fixed", "rot_x": parameters["rot_x"], "rot_y": parameters["rot_y"], "rot_z": parameters["rot_z"]}]
    return rotations
    
#def prepare_translations(parameters):
                

    
def render_all_patterns_and_rotations(parameters, patterns, rotations):
    scheduler = Scheduler.getInstance()
    scheduler.stop()
    pars = parameters

    if pars["cpu_count"] == -1:
        pars["cpu_count"] = multiprocessing.cpu_count()

    for i in range(0, pars["cpu_count"]):
        scheduler.registerWorker(LocalWorker(i,'wrk%i'%i))

    scheduler.start()
    queue = RenderQueue()
    
    # Iterate over all rotations
    for r_cnt, rotation in enumerate(rotations):
        res_path = os.path.join(pars["root_path"], pars["result_path"], rotation["name"])
        os.makedirs(res_path, exist_ok=parameters["result_overwrite"])

        # Iterate over all patterns
        for p_cnt, pattern in enumerate(patterns):
            param_map = map_render_parameters(pars, rotation, pattern)
            param_map["result_name"] = res_path + "/img_%03i"%p_cnt
            
            img = render_scene(scheduler, queue, pars, param_map)
            if not parameters["pattern_colored"]:
                img = rgb2gray(img)
                
            #print(img.dtype, img.shape)
            img = Image.fromarray(img.astype(np.uint8))
            img.save(param_map["result_name"] + ".png")
            
        # Render with ambient illumination
        param_map = map_render_parameters(pars, rotation, patterns[1]) # TODO proper selection of black pattern
        param_map["result_name"] = res_path + "/img_amb"
        param_map["const_radiance"] = "0.25"

        img = render_scene(scheduler, queue, pars, param_map)
        if not parameters["pattern_colored"]:
            img = rgb2gray(img)

        img = Image.fromarray(img.astype(np.uint8))
        img.save(param_map["result_name"] + ".png")


def map_render_parameters(pars, rotation, pattern):
    # TODO cleanup
    paramMap = StringMap()
    paramMap['pattern_name'] = pattern
    paramMap['object_name'] = os.path.join(pars["root_path"], pars["object_path"])


    
    for p in pars:
        if p == "focal_length":
            paramMap[p] = str(pars[p]) + "mm"
        if p == "obj_rx" or p == "obj_ry" or p == "obj_rz":
            a = np.array([[0, pars["obj_rx"], pars["obj_ry"]],
                          [-pars["obj_rx"], 0, pars["obj_rz"]],
                          [-pars["obj_ry"], -pars["obj_rz"], 0]])
            cayley = (np.eye(3) + a).dot(np.linalg.inv(np.eye(3) - a))
            t = np.vstack([cayley, np.zeros(3)])
            x = np.array([[0, 0, 0, 1]])
            trans = np.hstack([t, x.T]).reshape(-1)
            #print(trans)
            tstr = ""
            for t in trans:
                tstr += "%0.6f "%t
            paramMap["obj_transform2"] = tstr
        if p == "proj_rx" or p == "proj_ry" or p == "proj_rz":
            a = np.array([[0, pars["proj_rx"], pars["proj_ry"]],
                          [-pars["proj_rx"], 0, pars["proj_rz"]],
                          [-pars["proj_ry"], -pars["proj_rz"], 0]])
            cayley = (np.eye(3) + a).dot(np.linalg.inv(np.eye(3) - a))
            t = np.vstack([cayley, np.zeros(3)])
            x = np.array([[0, 0, 0, 1]])
            trans = np.hstack([t, x.T]).reshape(-1)
            #print(trans)
            tstr = ""
            for t in trans:
                tstr += "%0.6f "%t
            paramMap["proj_transform2"] = tstr
        if p == "obj_trans":
            tstr = ""
            trans = pars[p]
            for t in trans:
                tstr += "%0.6f "%t
            paramMap["obj_transform"] = tstr
        if p == "proj_trans":
            tstr = ""
            trans = pars[p]
            for t in trans:
                tstr += "%0.6f "%t
            paramMap["proj_transform"] = tstr
        paramMap[p] = str(pars[p])
        
    paramMap['rot_x'] = str(rotation["rot_x"])
    paramMap['rot_y'] = str(rotation["rot_y"])
    paramMap['rot_z'] = str(rotation["rot_z"])
    return paramMap
            
def render_scene(scheduler, queue, pars, param_map):
    fileResolver = Thread.getThread().getFileResolver()
    fileResolver.appendPath('rdata')

    scene_path = os.path.join(pars["root_path"], pars["scene_path"])
    scene = SceneHandler.loadScene(fileResolver.resolve(scene_path), param_map)
    scene.setDestinationFile(param_map["result_name"])
    job=RenderJob('myRenderJob', scene, queue)
    job.start()

    queue.waitLeft(0)
    film=scene.getFilm()
    size=film.getSize()
    bitmap=Bitmap(Bitmap.ERGB, Bitmap.EUInt8, size)
    film.develop(Point2i(0,0), size, Point2i(0,0), bitmap)
    res = np.array(bitmap.buffer())
    return res

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
    
def prepare_patterns(parameters):
    pattern_path = os.path.join(parameters["root_path"], parameters["pattern_path"])
    calib_path = os.path.join(parameters["root_path"], parameters["pattern_path"], "calibrated")
    if parameters["pattern_numbered"]:
        pattern_images = sorted(glob.glob(pattern_path + "/*[0-9].png"))
    else:
        pattern_images = sorted(glob.glob(pattern_path + "/*.png"))
    
    os.makedirs(calib_path, exist_ok=True)
    
    render_patterns = []
    
    for pi in pattern_images:
        assert os.path.isfile(pi)
        stem = pi.split("/")[-1]
        
        tex = Image.open(pi)
        if not parameters["pattern_colored"]:
            tex = tex.convert('L')
        
        tex = np.array(tex)
        tex_height, tex_width = tex.shape[0], tex.shape[1]
        if len(tex.shape) == 3:
            channels = tex.shape[2]
        else:
            channels = 1
        
        # TODO allow for larger patterns as well
        assert tex_height == parameters["pattern_height"] and tex_width == parameters["pattern_width"]
        
        if parameters["pattern_calibrate"]:
            pass
            # TODO: Apply distortion and calibration
            
        if parameters["pattern_quadrify"]:
            if tex_width > tex_height:
                if channels > 1:
                    z = np.zeros((tex_width-tex_height, tex_width, channels), dtype="uint8")
                else:
                    z = np.zeros((tex_width-tex_height, tex_width), dtype="uint8")                    
                if parameters["pattern_pad_below"]:
                    tex = np.concatenate((tex, z), axis=0)
                else:
                    tex = np.concatenate((z, tex), axis=0)

        if parameters["pattern_overwrite"] or not os.path.isfile(calib_path + "/" + stem.replace(".png", "_calib.png")):
            tex = Image.fromarray(tex)
            tex.save(calib_path + "/" + stem.replace(".png", "_calib.png"))
                
        render_patterns.append(calib_path + "/" + stem.replace(".png", "_calib.png"))

    assert len(render_patterns) == len(pattern_images)
    return render_patterns


    
#render_object("rendering/configs/parameters.json")
#render_object("rendering/configs/parameters2.json")
render_object("rendering/configs/parameters3.json")

46 19 [{'name': 'rot_000', 'rot_x': 0, 'rot_y': -90.0, 'rot_z': 0}, {'name': 'rot_001', 'rot_x': 0, 'rot_y': -80.0, 'rot_z': 0}, {'name': 'rot_002', 'rot_x': 0, 'rot_y': -70.0, 'rot_z': 0}, {'name': 'rot_003', 'rot_x': 0, 'rot_y': -60.0, 'rot_z': 0}, {'name': 'rot_004', 'rot_x': 0, 'rot_y': -50.0, 'rot_z': 0}, {'name': 'rot_005', 'rot_x': 0, 'rot_y': -40.0, 'rot_z': 0}, {'name': 'rot_006', 'rot_x': 0, 'rot_y': -30.0, 'rot_z': 0}, {'name': 'rot_007', 'rot_x': 0, 'rot_y': -20.0, 'rot_z': 0}, {'name': 'rot_008', 'rot_x': 0, 'rot_y': -10.0, 'rot_z': 0}, {'name': 'rot_009', 'rot_x': 0, 'rot_y': 0.0, 'rot_z': 0}, {'name': 'rot_010', 'rot_x': 0, 'rot_y': 10.0, 'rot_z': 0}, {'name': 'rot_011', 'rot_x': 0, 'rot_y': 20.0, 'rot_z': 0}, {'name': 'rot_012', 'rot_x': 0, 'rot_y': 30.0, 'rot_z': 0}, {'name': 'rot_013', 'rot_x': 0, 'rot_y': 40.0, 'rot_z': 0}, {'name': 'rot_014', 'rot_x': 0, 'rot_y': 50.0, 'rot_z': 0}, {'name': 'rot_015', 'rot_x': 0, 'rot_y': 60.0, 'rot_z': 0}, {'name': 'rot_016', 'rot_

In [31]:
import json

pars = {
  # Object Rotation parameters
  "rot_x": 0,
  "rot_y": 0,
  "rot_z": 0,
  "rot_range": [0, 360, 3],
  "rot_type": "Fixed", #"Turntable", #"Fixed", "Random"
  # Object Translation parameters
  "trans_x": 0.0,
  "trans_y": 0.0,
  "trans_z": 0.0,
  # Camera parameters
  "cam_aperture": 0.00016,
  "cam_focus": 0.95,
  "cam_fovy": 18.0,
  "cam_tx": 0.0,
  "cam_ty": 0.0,
  "cam_tz": 0.0,
  "cam_ux": 0.0,
  "cam_uy": 1.0,
  "cam_uz": 0.0,
  "cam_x": 0.0,
  "cam_y": 0.0,
  "cam_z": -0.95,
  "cam_height": 2048,
  "cam_width": 2048,
  # Global parameters
  "cpu_count": 10,
  "samples": 128,
  "result_path": "results/obj_ulp2",
  "result_overwrite": True,
  "scene_path": "scenes/scene_obj.xml",
  "root_path": "rendering",
  # Pattern parameters
  "pattern_path": "patterns/ulp/0.10_40",#mps/64-05",
  "pattern_numbered": True, # If patterns have numbered names
  "pattern_colored": False, # Colored patterns/rendering
  "pattern_calibrate": True, # Calibrate/predistort patterns
  "pattern_overwrite": True, # Overwrite possible existing patterns
  "pattern_quadrify": True, # Pad patterns for quadratic spotlight
  "pattern_pad_below": False,
  "pattern_width": 1920,
  "pattern_height": 1080,
  # Object parameters
  "object_path": "objects/st_4_40k_occ_nurbs.ply",
  "obj_rx": 0.0,
  "obj_ry": 0.0,
  "obj_rz": 0.0,
  "obj_ttx": 0.0,
  "obj_tty": 0.0,
  "obj_ttz": 0.0,
  "obj_x": 0.0,
  "obj_y": 0.0,
  "obj_z": 0.0,
  # Projector parameters
  "pro_aperture": 0.0003,
  "pro_beamwidth": 18.415,
  "pro_cutoff": 18.415,
  "pro_focus": 0.4,
  "pro_intensity": 0.5,
  "pro_offset_x": 0.5,
  "pro_offset_y": 0.5,
  "pro_scale_x": -0.5,
  "pro_scale_y": 0.5,
  "pro_tx": -0.01,
  "pro_ty": 0.0,
  "pro_tz": 0.0,
  "pro_ux": 0.0,
  "pro_uy": 1.0,
  "pro_uz": 0.0,
  "pro_x": -0.1,
  "pro_y": 0.0,
  "pro_z": -0.35,
  "pro_ox": 0.0,
  "pro_oy": 0.0,
  "pro_oz": 0.0,
  # Lighting parameters
  "const_radiance": 0.0
}

with open("rendering/configs/parameters.json", "w") as fi:
    json.dump(pars, fi, indent=2, sort_keys=True)

In [116]:
import numpy as np
import json

# Object location from sls scanner
r = np.array([[0.01661032,  0.99959023, 0.02325158],
                [0.86490919, -0.026032, 0.50125285],
                [0.50019783,  0.01182833, -0.86583037]])

ria = np.array([[1, 0, 0.0],
              [0, 1, 0.0],
              [0, 0, 1.0]])

t = np.array([[0.0, -0.02, 0.91, 1.0]])
r = np.vstack([ria.T, np.zeros(3)])
trans = np.hstack([r, t.T]).reshape(-1)

# Projector rotation and translation
r = np.array([[0.87953157, -0.01611216,  0.47556768],
             [0.01206672,  0.99986039,  0.01155851],
             [-0.47568752, -0.00442754,  0.8796032]])

#print(np.linalg.norm(r, axis=1))
t = np.array([[0.22634958146, 0.07420267954, 0.4120573676, 1.0]])
r = np.vstack([r.T, np.zeros(3)])
trans2 = np.hstack([r, t.T]).reshape(-1)

size = "medium" #small

if size == "large":
    cam_w = 6464
    cam_h = 4852
    suffix = ""
if size == "medium":
    cam_w = int(6274/2)
    cam_h = int(4682/2)
    suffix = "_m"
if size == "small":
    cam_w = int(6274/4)
    cam_h = int(4682/4)
    suffix = "_s"


pars = {
  # Object Rotation parameters
  "rot_x": 0,
  "rot_y": 0,
  "rot_z": 0,
  "rot_range": [-90, 100, 19],
  "rot_type": "Turntable", #"Turntable", #"Fixed", "Random"
  # Object Translation parameters
  "trans_x": 0.0,
  "trans_y": 0.0,
  "trans_z": 0.0,
  # Camera parameters
  "cam_aperture": 0.00016,
  "cam_focus": 0.8,
  "cam_fovy": 18.0,
  "cam_tx": 0.0,
  "cam_ty": 0.0,
  "cam_tz": 1.0,
  "cam_ux": 0.0,
  "cam_uy": -1.0,
  "cam_uz": 0.0,
  "cam_x": 0.0,
  "cam_y": 0.0,
  "cam_z": 0.0,
  "cam_height": cam_h,
  "cam_width": cam_w,
  # Global parameters
  "cpu_count": 10,
  "samples": 128,
  "result_path": "results/obj_gray_medium",
  "result_overwrite": True,
  "scene_path": "scenes/scene_sls.xml",
  "root_path": "rendering",
  # Pattern parameters
  "pattern_path": "patterns/gray/default",#mps/64-05",
  "pattern_numbered": True, # If patterns have numbered names
  "pattern_colored": False, # Colored patterns/rendering
  "pattern_calibrate": True, # Calibrate/predistort patterns
  "pattern_overwrite": True, # Overwrite possible existing patterns
  "pattern_quadrify": True, # Pad patterns for quadratic spotlight
  "pattern_pad_below": True,
  "pattern_width": 1920,
  "pattern_height": 1080,
  # Object parameters
  "object_path": "objects/st_4_40k_occ_c.ply",
  "obj_rx": 0.0,
  "obj_ry": 0.0,
  "obj_rz": 0.0,
  "obj_ttx": 0.0,
  "obj_tty": 0.0,
  "obj_ttz": 0.0,
  "obj_x": 0.0,
  "obj_y": 0.0,
  "obj_z": 0.0,
  "obj_trans": trans.tolist(),
  # Projector parameters
  "pro_aperture": 0.0002,
  "pro_beamwidth": 18.415,
  "pro_cutoff": 18.467,
  "pro_focus": 0.5,
  "pro_intensity": 0.5,
  "pro_offset_x": 0.457,
  "pro_offset_y": 0.003,
  "pro_scale_x": 0.5,
  "pro_scale_y": 0.5,
  "pro_tx": -0.01,
  "pro_ty": 0.0,
  "pro_tz": 0.0,
  "pro_ux": 0.0,
  "pro_uy": 1.0,
  "pro_uz": 0.0,
  "pro_x": -0.1,
  "pro_y": 0.0,
  "pro_z": -0.35,
  "pro_ox": 0.0,
  "pro_oy": 0.0,
  "pro_oz": 0.0,
  "proj_trans": trans2.tolist(),
  # Lighting/Material parameters
  "const_radiance": 0.0,
  "mat_ior": 1.0634878252015734,
  "mat_alpha": 0.22150848706292028,
  "mat_diff": 0.9112623909906508
}


# Calibration
# pars['scene_path'] = "scenes/scene_sls_cal.xml"
# pars['result_path'] = "results/calib_medium"
# pars['object_path'] = "objects/cb10_8_3_ms_c.ply" 
# pars["rot_range"] = [[-30, 30], [-30, 30], [-30, 30], 5]
# pars["rot_type"] = "Random" #"Turntable", #"Fixed", "Random"
# pars["const_radiance"] = 0.05
# pars["pro_intensity"] = 0.5


with open("rendering/configs/parameters3.json", "w") as fi:
    json.dump(pars, fi, indent=2, sort_keys=True)